In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bengkulu,ID,2020-04-18 02:06:39,-3.80,102.27,84.81,68,77,4.85
1,1,Hobart,AU,2020-04-18 02:01:55,-42.88,147.33,55.40,43,40,14.99
2,2,Nikolskoye,RU,2020-04-18 02:06:40,59.70,30.79,37.40,89,75,15.66
3,3,Zyryanka,RU,2020-04-18 02:06:40,65.75,150.85,5.22,92,100,20.49
4,4,Kenai,US,2020-04-18 02:06:40,60.55,-151.26,48.20,70,90,3.44


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [23]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [21]:
# Heatmap of temperature
# Gets the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature
# Assigns the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.6)



In [27]:
# Assigns the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Adds the heatmap layer
fig.add_layer(heat_layer)

# Calls the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)
# Calls the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)
# Calls the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = wind, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)
# Calls the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Prompts customer for info on min and max temp for trip
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [37]:
# uses loc to filter out the data according to this condition 
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bengkulu,ID,2020-04-18 02:06:39,-3.80,102.27,84.81,68,77,4.85
5,5,Kapaa,US,2020-04-18 02:06:40,22.08,-159.32,80.60,61,1,11.41
6,6,Port Macquarie,AU,2020-04-18 02:05:06,-31.43,152.92,75.00,56,100,1.01
12,12,Sumenep,ID,2020-04-18 02:06:42,-7.02,113.87,86.25,72,12,8.59
13,13,Rikitea,PF,2020-04-18 02:06:42,-23.12,-134.97,77.36,63,1,1.99


In [35]:
preferred_cities_df.count()

City_ID       160
City          160
Country       160
Date          160
Lat           160
Lng           160
Max Temp      160
Humidity      160
Cloudiness    160
Wind Speed    160
dtype: int64

In [42]:
# creates copy of the preferred cities df
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bengkulu,ID,84.81,-3.80,102.27,
5,Kapaa,US,80.60,22.08,-159.32,
6,Port Macquarie,AU,75.00,-31.43,152.92,
12,Sumenep,ID,86.25,-7.02,113.87,
13,Rikitea,PF,77.36,-23.12,-134.97,


In [43]:
# Sets parameters to search for a hotel for url
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [48]:

# Iterates through dataframe using iterrows()
for index, row in hotel_df.iterrows():
    # Gets the latitude and longitude for particular row and stores it
    lat = row["Lat"]
    lng = row["Lng"]

    # Adds the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # Uses the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Makes request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grabs the first hotel from the results and stores the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [51]:
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bengkulu,ID,84.81,-3.80,102.27,Grage Hotel Bengkulu
5,Kapaa,US,80.60,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
6,Port Macquarie,AU,75.00,-31.43,152.92,Rydges Port Macquarie
12,Sumenep,ID,86.25,-7.02,113.87,OYO 3271 Homestay Pandian
13,Rikitea,PF,77.36,-23.12,-134.97,Pension Maro'i
15,Axim,GH,80.69,4.87,-2.24,AXIM BEACH RESORT AND SPA
16,Puerto Ayora,EC,77.00,-0.74,-90.35,Finch Bay Galapagos Hotel
22,Saint-Philippe,RE,75.20,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
25,Atuona,PF,82.35,-9.80,-139.03,Villa Enata
26,Inhambane,MZ,75.00,-23.86,35.38,MHL Turismo SA


In [54]:
# Adds heatmap as well as markers for the hotels
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
# adds the information for the hotel name, city, country, and max temp
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Adds heatmap as well as markers for the hotels
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))